## This code codebook serves multiple purposes

- Join counselors' role and login time and calculate their accumulative hours on duty when handling that particular session
- Exclude outliers with missing login time
- Regression analysis on attention score along with counselor's roles, counselor's accumulative hours on duty, users visiting hours and downstream outcomes.

@author: Yucan Xu (chicoxu@connect.hku.hk)

In [ ]:
import pandas as pd
import numpy as np
import glob,os,re,string
import pytz
import datetime
import pymongo
import re
from datetime import timedelta
from pingouin import ttest
import researchpy as rp
from scipy.stats import pearsonr
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Read and join counselors' info and login time from database

In [ ]:
# try:
#     # password authentication to login to MongoDB
#     client = pymongo.MongoClient(
#     )
#     # access Openup Database and authenticate
#     db = client['OpenupDB']
#     print(f'Collections: {db.list_collection_names()}')

# except:
#     print('Cannot access OpenUp database.')
#     if client:
#         client.close()

In [ ]:
log_records = pd.DataFrame(list(db['OauthToken'].find()))

In [ ]:
log_records

In [ ]:
d=timedelta(hours=8)

In [ ]:
## check na

In [ ]:
log_records[log_records['loginDateTime'].isna()]

In [ ]:
log_records_drop = log_records[(log_records['loginDateTime'].notna()) & 
                               (log_records['logoutDateTime'].notna())].copy()

In [ ]:
log_records_drop['shift'] = log_records_drop['logoutDateTime'] - log_records_drop['loginDateTime']

In [ ]:
log_records_drop2 = log_records_drop[log_records_drop['shift']<timedelta(hours=24)].copy()
log_records_drop2.shape

In [ ]:
log_records_drop2['loginDateTime'] = log_records_drop2['loginDateTime'] + d

In [ ]:
pred = pd.read_csv('premature_depature_detected_20-21.csv')

In [ ]:
login_times = []
counselors = []
for i in pred.index:
    counselor_id = pred.loc[i]['counselorId1']
    start_datetime = pred.loc[i]['startDateTime']
    counselor_records = log_records_drop2[log_records_drop2['systemId'] == counselor_id].copy()
    login_time = counselor_records[counselor_records['loginDateTime']<=pred.loc[i]['startDateTime']].drop_duplicates(
        subset=['systemId'],keep='last')['loginDateTime']
    login_times.append(login_time)
    counselor_role = counselor_records[counselor_records['loginDateTime']<=pred.loc[i]['startDateTime']].drop_duplicates(
        subset=['systemId'],keep='last')['type']
    counselors.append(counselor_role)

In [ ]:
pred['login_time'] = login_times

In [ ]:
pred['counselor_role'] = counselors

In [ ]:
pred['login_time'] = pred['login_time'].apply(lambda x:x.values)

In [ ]:
pred['counselor_role'] = pred['counselor_role'].apply(lambda x:x.values)

In [ ]:
len(pred['login_time'][0])

## Exclude sessions with login time error

In [ ]:
pd_drop = pred[pred['login_time'].map(len)==1].copy()

In [ ]:
pd_drop['login_time'] = pd_drop['login_time'].apply(lambda x:x[0])

In [ ]:
pd_drop.columns

In [ ]:
pd_drop.login_time = pd.to_datetime(pd_drop.login_time)

In [ ]:
pd_drop.startDateTime = pd.to_datetime(pd_drop.startDateTime)

In [ ]:
pd_drop['counselor_onduty_hours'] = pd_drop['startDateTime'] - pd_drop['login_time']

In [ ]:
pd_drop['startDateTime'][0]

## Exlucde outliers that counselor_onduty_hours >=1 day

In [ ]:
pd_drop['counselor_onduty_hours'].min()

In [ ]:
concurrent = pd.read_csv('concurrent_master_join.csv')

In [ ]:
concurrent_join = concurrent.join(
    pd_drop[['conversationId','counselor_onduty_hours']].set_index(
    'conversationId'),on='conversationId',how='left')

In [ ]:
concurrent_drop = concurrent_join[concurrent_join['counselor_onduty_hours']<=timedelta(hours=24)].copy()

In [ ]:
concurrent_drop['hours_onduty'] = concurrent_drop['counselor_onduty_hours'].dt.seconds/3600

In [ ]:
concurrent_drop['hours_onduty'].max()

In [ ]:
concurrent_drop.columns

In [ ]:
concurrent_drop = concurrent_drop.rename(columns={
    '1. 比起對話前，你現在的感覺如何?.1':'Q1',
    '3. 你認為本對話服務對你有多大幫助？[1至5選一]':'Q3',
    '7. 如果你的朋友有需要，你會向他推薦我們的服務嗎？':'Q7'
})

In [ ]:
concurrent_drop

In [ ]:
concurrent_drop2 = concurrent_drop.join(pd.get_dummies(concurrent_drop[['role','visting_period']]))

In [ ]:
concurrent_drop2.columns

In [ ]:
concurrent_drop2 = concurrent_drop2.drop(columns=['role_Volunteer','visting_period_1-8'])

## Regression on Q1

In [ ]:
Q1 = concurrent_drop2[concurrent_drop2['Q1'].notna()].copy()

In [ ]:
Q1.shape

In [ ]:
x = sm.add_constant(Q1[['attention_score','hours_onduty',
                                   'role_Paid_staff','visting_period_17-24','visting_period_9-16']], prepend=False)

In [ ]:
y = Q1['Q1']

In [ ]:
mod = sm.OLS(y, x)

In [ ]:
res = mod.fit()

In [ ]:
print(res.summary2())

## Regression on Q3

In [ ]:
Q3 = concurrent_drop2[concurrent_drop2['Q3'].notna()].copy()

In [ ]:
Q3.shape

In [ ]:
x = sm.add_constant(Q3[['attention_score','hours_onduty',
                                   'role_Paid_staff','visting_period_17-24','visting_period_9-16']], prepend=False)

In [ ]:
y = Q3['Q3']

In [ ]:
mod = sm.OLS(y, x)

In [ ]:
res = mod.fit()

In [ ]:
print(res.summary2())

## Regression on Q7

In [ ]:
Q7 = concurrent_drop2[concurrent_drop2['Q7'].notna()].copy()

In [ ]:
Q7.shape

In [ ]:
x = sm.add_constant(Q7[['attention_score','hours_onduty',
                                   'role_Paid_staff','visting_period_17-24','visting_period_9-16']], prepend=False)

In [ ]:
logistic_model = sm.GLM(Q7[['Q7']], x,
                        family=sm.families.Binomial())

In [ ]:
results = logistic_model.fit()

In [ ]:
print(results.summary())

In [ ]:
params = results.params
conf = results.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
np.exp(conf)

## Regression on Premature departure

In [ ]:
concurrent_drop2.columns

In [ ]:
concurrent_drop2.shape

In [ ]:
x = sm.add_constant(concurrent_drop2[['attention_score','hours_onduty',
                                   'role_Paid_staff','visting_period_17-24','visting_period_9-16']], prepend=False)

In [ ]:
logistic_model = sm.GLM(concurrent_drop2[['premature_departure_predicted']], x,
                        family=sm.families.Binomial())

In [ ]:
results = logistic_model.fit()

In [ ]:
print(results.summary())

In [ ]:
params = results.params
conf = results.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
np.exp(conf)